# Data Preprocessing

- We need to import and transform following MIMIC-III datasets:
  
  DIAGNOSES_ICD.csv: Each row in this file maps Hospitalization ID (HADM_ID) of a patient with a unique ICD_9 CODE.
  
  Example:
  
  |ROW_ID|SUBJECT_ID|**HADM_ID**|SEQ_NUM|**ICD9_CODE**|
  |--------|------------|---------|---------|-----------|
  |1297|109|172335|1|"0030"|
  |1299|109|172335|3|"0038"|
  |1301|109|173633|5|"0031"|
  
  
  The aim is to transform this dataset to dataframe DIGNOSES with index as HADM_ID and columns as unique ICD_9 codes (6984 in total), to represent multi-hot encoding of ICD_9 codes for given hospitalization.
  
  |HADM_ID|ICD9_CODE_0030|ICD9_CODE_0031|ICD9_CODE_0038|
  |-------|--------------|--------------|--------------|
  |172335|1|0|1|
  |173633|0|1|0|
  
  
  
  NOTEEVENTS.csv: Each row maps HADM_ID (Hospitalization ID) with a free text Discharge summary (TEXT) field.
  
  |ROW_ID|SUBJECT_ID|**HADM_ID**|CHARTDATE|CHARTTIME|STORETIME|CATEGORY|DESCRIPTION|CGID|ISERROR|**TEXT**|
  |------|----------|-----------|---------|---------|---------|--------|-----------|----|-------|--------|
  |174|22532|167853|2151-08-04|||Discharge summary|Report|||Admission Date:  [\*\*2151-7-16**]       Discharge Date:  [\*\*2151-8-4**] Service: ADDENDUM: RADIOLOGIC STUDIES:  Radiologic studies also included a chest| 
 
  The aim is to transform this to dataframe with HADM_ID as index and TEXT as column.
  |HADM_ID|TEXT|
  |-------|----|
  |167853|Admission Date:  [\*\*2151-7-16**]       Discharge Date:  [\*\*2151-8-4**] Service: ADDENDUM: RADIOLOGIC STUDIES:  Radiologic studies also included a chest|

<br/>
<br/>

- The processed dataframes are then stored (as csv) for further usage.

- This notebook uses Rapids framework (cudf and dask dataframe) to enable faster processing of Pandas dataframe on GPU.

#### Check GPU Version

In [2]:
# Check GPU
!nvidia-smi

Sat Apr 15 19:43:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.78.01    Driver Version: 525.78.01    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8     2W /  50W |      3MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
This set up script:

1. Checks to make sure that the GPU is RAPIDS compatible
1. Installs the **current stable version** of RAPIDSAI's core libraries using pip, which are:
  1. cuDF
  1. cuML
  1. cuGraph
  1. xgboost

**This will complete in about 3-4 minutes**

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 385 (delta 86), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (385/385), 105.74 KiB | 2.20 MiB/s, done.
Resolving deltas: 100% (188/188), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 2.7 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.

## Critical Imports

In [10]:
# Critical imports
import cudf
import cuml
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [ ]:
# Mount the project directory in Google drive. (Its only intended to be run in colab environment.)

from google.colab import drive
drive.mount('drive')

In [ ]:
# Define the base project directory.

PROJECT_DIR = 'drive/My Drive/cs598-dl/' # For Google drive only

# PROJECT_DIR = '../' # For local directory

In [ ]:
# We first process MIMIC-III DIAGNOSES_ICD dataset.

In [11]:
# Read DIAGNOSES_ICD.csv from data directory, and pre-process.

diagnoses_df = pd.read_csv(PROJECT_DIR + 'data/DIAGNOSES_ICD.csv', usecols=['HADM_ID', 'ICD9_CODE'])
diagnoses_df = diagnoses_df.astype({'ICD9_CODE': 'string'})

# Collect all unique ICD_9 codes and create new DataFrame codes_df.
codes_df = pd.DataFrame(diagnoses_df['ICD9_CODE'].unique(), columns = ['ICD9_CODE'])

# Create DataFrame representing one-hot encoding of ICD_9 codes.
one_hot_enc_df = pd.get_dummies(codes_df, columns = ['ICD9_CODE'], dtype='bool')

# Join codes_df and one_hot_enc_df, based on index
codes_df = codes_df.join(one_hot_enc_df)

# Next we merge diagnoses_df and codes_df, to create our final form mapping each HADM_ID with multi-hot encoding of ICD_9 codes.
# This is very heavy operation due to large number of rows in diagnoses_df and large number of columns in codes_df.
# So we utilize Dask DataFrame to parallelize this operation on GPU cores.

# Create Dask DataFrame from codes_df for distributed processing in GPU.
codes_df = dd.from_pandas(codes_df, npartitions = 10)

# Create Dask DataFrame from diagnoses_df for distributed processing in GPU.
diagnoses_df = dd.from_pandas(diagnoses_df, npartitions = 10)

# Merge diagnoses_df and codes_df based on column 'ICD9_CODE'. 
# Dask operations are lazy and do not materialize until 'compute()' method is invoked.
diagnoses_df = diagnoses_df.merge(codes_df, on='ICD9_CODE').compute()

diagnoses_df = diagnoses_df.drop(['ICD9_CODE'], axis = 1)

# This step will group all ICD_9 codes corresponding to a given HADM_ID and build a multi-hot embedding.
diagnoses_df = diagnoses_df.groupby('HADM_ID').any().reset_index()

print(diagnoses_df)

AttributeError: module 'time' has no attribute 'start'

In [13]:
import time

# Read DIAGNOSES_ICD.csv from data directory, and pre-process.

diagnoses_df = pd.read_csv(PROJECT_DIR + 'data/DIAGNOSES_ICD.csv', usecols=['HADM_ID', 'ICD9_CODE'])
diagnoses_df = diagnoses_df.astype({'ICD9_CODE': 'string'})

# Collect all unique ICD_9 codes and create new DataFrame codes_df.
codes_df = pd.DataFrame(diagnoses_df['ICD9_CODE'].unique(), columns = ['ICD9_CODE'])

# Create DataFrame representing one-hot encoding of ICD_9 codes.
one_hot_enc_df = pd.get_dummies(codes_df, columns = ['ICD9_CODE'], dtype='bool')

# Join codes_df and one_hot_enc_df, based on index
codes_df = codes_df.join(one_hot_enc_df)

sta = time.time()
# Merge diagnoses_df and codes_df based on column 'ICD9_CODE'. 
# Dask operations are lazy and do not materialize until 'compute()' method is invoked.
diagnoses_df = diagnoses_df.merge(codes_df, on='ICD9_CODE')
end = time.time()
print(end - sta)

diagnoses_df = diagnoses_df.drop(['ICD9_CODE'], axis = 1)

# This step will group all ICD_9 codes corresponding to a given HADM_ID and build a multi-hot embedding.
diagnoses_df = diagnoses_df.groupby('HADM_ID').any().reset_index()

print(diagnoses_df)

14.143898010253906
       HADM_ID  ICD9_CODE_0030  ICD9_CODE_0031  ICD9_CODE_0038  \
0       100001           False           False           False   
1       100003           False           False           False   
2       100006           False           False           False   
3       100007           False           False           False   
4       100009           False           False           False   
...        ...             ...             ...             ...   
58971   199993           False           False           False   
58972   199994           False           False           False   
58973   199995           False           False           False   
58974   199998           False           False           False   
58975   199999           False           False           False   

       ICD9_CODE_0039  ICD9_CODE_0041  ICD9_CODE_0048  ICD9_CODE_0049  \
0               False           False           False           False   
1               False           False     

In [ ]:
# Next we process MIMIC-III NOTEEVENTS dataset.

In [ ]:
# Import dataset and pre-process.
notes_df = pd.read_csv('drive/My Drive/cs598-dl/data/NOTEEVENTS.csv', usecols=['HADM_ID', "CATEGORY","DESCRIPTION", "TEXT"])
notes_df = notes_df.dropna()

# Only filter-in notes which are 'Discharge summary' and are of sub-type 'Report'.
notes_df = notes_df[(notes_df['CATEGORY'] == 'Discharge summary') & (notes_df['DESCRIPTION'] == 'Report')]
notes_df = notes_df.drop(['CATEGORY', 'DESCRIPTION'], axis=1)
notes_df = notes_df.astype({'HADM_ID': 'int64'})
notes_df = notes_df.drop_duplicates(subset = 'HADM_ID')
print(notes_df)

       HADM_ID                                               TEXT
0       167853  Admission Date:  [**2151-7-16**]       Dischar...
1       107527  Admission Date:  [**2118-6-2**]       Discharg...
2       167118  Admission Date:  [**2119-5-4**]              D...
3       196489  Admission Date:  [**2124-7-21**]              ...
4       135453  Admission Date:  [**2162-3-3**]              D...
...        ...                                                ...
55970   147266  Admission Date:  [**2147-2-25**]              ...
55971   129802  Admission Date:  [**2190-5-13**]              ...
55972   182558  Admission Date:  [**2121-6-13**]              ...
55973   184741  Admission Date:  [**2182-4-19**]              ...
55974   121964  Admission Date:  [**2186-6-16**]              ...

[55102 rows x 2 columns]


In [5]:
# We next select the subset of rows in diagnoses_df and notes_df with common set of HADM_IDs, 
# and remove other rows from each DataFrame. Such rows can not be used in training or testing.

In [ ]:
# Collect all hadm_ids from diagnoses_df
hadm_ids_from_diagnoses_df = diagnoses_df.filter(items = ['HADM_ID'])

# Collect all hadm_ids from diagnoses_df
hadm_ids_from_notes_df = notes_df.filter(items = ['HADM_ID'])

# Generate DataFrame with common set of HADM_IDs.
hadm_ids_df = hadm_ids_from_diagnoses_df.merge(hadm_ids_from_notes_df, how = 'inner')

# Filter rows in daignoses_df by merging with DataFrame containing common HADM_IDs.
diagnoses_df = diagnoses_df.merge(hadm_ids_df, on='HADM_ID', how = 'right')

# Similarly, filter rows in notes_df by merging with DataFrame containing common HADM_IDs.
notes_df = notes_df.merge(hadm_ids_df, on='HADM_ID', how = 'inner')

       HADM_ID
0       100001
1       100003
2       100006
3       100007
4       100009
...        ...
58971   199993
58972   199994
58973   199995
58974   199998
58975   199999

[58976 rows x 1 columns]


In [ ]:
# Pickle diagnoses_df
diagnoses_df.to_pickle(PROJECT_DIR + 'data/DIAGNOSES.pkl')

# Pickle notes_df
notes_df.to_pickle(PROJECT_DIR + 'data/NOTES.pkl')

In [ ]:
diagnoses_df = Da